# Derive Ratings Values from Source Data
This notebook provides a straight forward solution for gaining insight into the ratings across four different agencies. These are as follows:
- Moodys
- SP
- Fitch
- Internal Rating

This notebook takes the alphabetical values used by these agencies and converts them into a numeric format, which then allows for greater insight as to the highest and lowest ratings for a given stock across the four agencies, along with the average rating.

## Setup

In [31]:
# Import LUSID
import lusid
import lusid.models as models
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import format_instruments_response

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import pandas as pd
import json
import lusid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")
pd.set_option('display.max_columns', 500)

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")


print ('LUSID Environment Initialised')
print ('LUSID API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

instrument_scope = "ratingsDemo"

LUSID Environment Initialised
LUSID API Version:  0.6.10802.0


## Loading Data into Lusid
The intial data is loaded from a CSV file, this will then be upserted into Lusid. All of the stocks will be created as new instruments, with custom properties attached to allow for the various ratings.

In [2]:
vals = pd.read_csv("data/uk_stocks_with_ratings.csv")
vals

,Ticker,Name,Sector,ISIN,SEDOL,Figi,Moodys,SP,Fitch,Internal
0,III LN,3i,Financial Services,GB00B1YW4409,B1YW440,BBG000BZZ876,BBB+,A+,AA-,BBB
1,ADM LN,Admiral Group,Nonlife Insurance,GB00B02J6398,B02J639,BBG000PG2GZ0,NaN,A+,A,AA
2,AAL LN,Anglo American plc,Mining,GB00B1XZS820,B1XZS82,BBG000BBLDF4,BBB+,AA,AA,NaN
3,ANTO LN,Antofagasta,Mining,GB0000456144,45614,BBG000BD4SC9,AA,A-,A+,A+
4,AHT LN,Ashtead Group,Support Services,GB0000536739,53673,BBG000BD42C6,A,AA,BBB-,BBB+
...,...,...,...,...,...,...,...,...,...,...
95,ULVR LN,Unilever,Personal Goods,GB00B10RZP78,B10RZP7,BBG000C0M8X7,BBB,NaN,A,BBB
96,UU/ LN,United Utilities,"Gas, Water & Multi-utilities",GB00B39J2M42,B39J2M4,BBG000BBFLV5,BBB,BBB,AA,AA-
97,VOD LN,Vodafone Group,Mobile Telecommunications,GB00BH4HKS39,BH4HKS3,BBG000C6K6G9,NaN,BBB-,A,BBB
98,WTB LN,Whitbread,Retail hospitality,GB00B1KJJ408,B1KJJ40,BBG000BRVH05,BBB+,A,BBB,A-


In [3]:
# Create mapping for upsert
instrument_mapping = {
    "identifiers": {
        "Isin": "ISIN",
        "Sedol": "SEDOL",
        "Figi": "Figi"
    },
    "mapping_required": {
        "name": "Name",
        "definition.instrument_type": "$Equity",
        "definition.dom_ccy": "$GBP"
    },
    "property_columns": [
        {
            "source": "Moodys",
            "target": "Moodys Rating",
            "scope": "moodys",
        },
        {
            "source": "SP",
            "target": "SP Rating",
            "scope": "sp"
        },
        {
            "source": "Fitch",
            "target": "Fitch Rating",
            "scope": "fitch"
        },
        {
            "source": "Internal",
            "target": "Internal Rating",
            "scope": "internal_r"
        }
    ]
}

In [4]:
# Upsert to lusid
response = load_from_data_frame(
    api_factory=api_factory,
    scope=instrument_scope,
    data_frame=vals,
    mapping_required=instrument_mapping["mapping_required"],
    mapping_optional={},
    file_type="instrument",
    identifier_mapping=instrument_mapping["identifiers"],
    property_columns=instrument_mapping["property_columns"],
    properties_scope="ratings",
    instrument_scope=instrument_scope
)

instrument_response_objs =  [ value.lusid_instrument_id for key, value in response['instruments']['success'][0].values.items()]
#instrument_response_objs

## Create Derived Properties for Average Rating

In [141]:
ratings_mapping = """'AA'=90,
'AA-'=85,
'A+'=80,
'A'=75,
'A-'=70,
'BBB+'=65,
'BBB'=60,
'BBB-'=55
"""

fitch_rating = f"map(Properties[Instrument/fitch/FitchRating]: {ratings_mapping})"
moodys_rating = f"map(Properties[Instrument/moodys/MoodysRating]: {ratings_mapping})"
snp_rating = f"map(Properties[Instrument/sp/SPRating]: {ratings_mapping})"

In [53]:
derived_formula = f"""average(
coalesce({fitch_rating}, average(coalesce({moodys_rating}, {snp_rating}), coalesce({snp_rating}, {moodys_rating}))),
coalesce({moodys_rating}, average(coalesce({fitch_rating}, {snp_rating}), coalesce({snp_rating}, {fitch_rating}))),
coalesce({snp_rating}, average(coalesce({fitch_rating}, {moodys_rating}), coalesce({moodys_rating}, {fitch_rating})))
)
"""

derived_formula = derived_formula.replace("\n", "\t").replace(" ","")

In [56]:
# Create derived properties for ratings derivations: min, max, average
# TODO can I round and then convert back to letters?
try:
        
    properties_response = property_definitions_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope="ratings-scope",
            code="AverageRating",
            display_name="Average Rating",
            data_type_id=lusid.ResourceId(scope="system", code="number"),
            derivation_formula=derived_formula
        )
    )
    
    properties_response
except lusid.exceptions.ApiException as e:
        
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/ratings-scope/AverageRating' because it already exists.


## Create derived property for hierarchy

In [90]:
derived_formula = """
coalesce(Properties[Instrument/fitch/FitchRating], 
Properties[Instrument/moodys/MoodysRating], 
Properties[Instrument/sp/SPRating],
Properties[Instrument/internal_r/InternalRating],
'Unknown')
"""

derived_formula = derived_formula.replace("\n", "\t")

In [93]:
# Create derived properties for ratings derivations: min, max, average
# TODO can I round and then convert back to letters?
try:
        
    properties_response = property_definitions_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope="ratings-scope",
            code="PreferredRating",
            display_name="Preferred Rating",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            derivation_formula=derived_formula
        )
    )
    
    properties_response
except lusid.exceptions.ApiException as e:
    
    print(e)
        
    print(json.loads(e.body)["title"])

(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 13 Feb 2023 12:44:09 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4999', 'X-Rate-Limit-Reset': '2023-02-13T12:45:07.6462578Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMOCNS36TUU7:00000008', 'lusid-meta-correlationId': '0HMOCNS36TUU7:00000008', 'lusid-meta-duration': '1643', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagger.io; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-ur

## Average of top two

In [126]:
moodys_rating

"map(Properties[Instrument/moodys/MoodysRating]: 'AA'=90,\n    'AA-'=85,\n    'A+'=80,\n    'A'=75,\n    'A-'=70,\n    'BBB+'=65,\n    'BBB'=60,\n    'BBB-'=55\n)"

In [145]:
derived_formula = f"if(({fitch_rating} lte {moodys_rating}) and ({fitch_rating} lte {snp_rating})) then average({moodys_rating}, {snp_rating}) else -1"

derived_formula = derived_formula.replace("\n", "")
derived_formula

"if((map(Properties[Instrument/fitch/FitchRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55) lte map(Properties[Instrument/moodys/MoodysRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55)) and (map(Properties[Instrument/fitch/FitchRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55) lte map(Properties[Instrument/sp/SPRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55))) then average(map(Properties[Instrument/moodys/MoodysRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55), map(Properties[Instrument/sp/SPRating]: 'AA'=90,'AA-'=85,'A+'=80,'A'=75,'A-'=70,'BBB+'=65,'BBB'=60,'BBB-'=55)) else -1"

In [146]:
# Create derived properties for ratings derivations: min, max, average
# TODO can I round and then convert back to letters?
try:
        
    properties_response = property_definitions_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope="ratings-scope",
            code="AverageTopTwo",
            display_name="Average of Top Two",
            data_type_id=lusid.ResourceId(scope="system", code="number"),
            derivation_formula=derived_formula
        )
    )
    
    properties_response
except lusid.exceptions.ApiException as e:
    
    print(e)
        
    print(json.loads(e.body)["title"])

(400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 13 Feb 2023 14:37:05 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4999', 'X-Rate-Limit-Reset': '2023-02-13T14:38:05.0060545Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMODBOFK8ECL:00000008', 'lusid-meta-correlationId': '0HMODBOFK8ECL:00000008', 'lusid-meta-duration': '119', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagger.io; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri

## View instruments

In [94]:
instruments_properties_response = api_factory.build(
        lusid.api.InstrumentsApi
    ).get_instruments(
        scope=instrument_scope,
        identifier_type="LusidInstrumentId",
        request_body=instrument_response_objs,
        property_keys=[
            "Instrument/fitch/FitchRating",
            "Instrument/moodys/MoodysRating",
            "Instrument/sp/SPRating",
            "Instrument/internal_r/InternalRating",
            "Instrument/ratings-scope/AverageRating",
            "Instrument/ratings-scope/PreferredRating"
        ]
    )

In [95]:
def instrument_response_to_df(response):
    
    df_rows = []
    
    for item in response.values.values():
        
        row = {}
        row["isin"] = item.identifiers["Isin"]
        row["name"] = item.name
    
        for prop in item.properties:
            
            if prop.value.label_value:
                
                row[prop.key] = prop.value.label_value
                
            else:
                
                 row[prop.key] = prop.value.metric_value.value
        
        df_rows.append(row)
                
    return pd.DataFrame(df_rows)
    
    

In [96]:
instrument_response_to_df(instruments_properties_response)

,isin,name,Instrument/fitch/FitchRating,Instrument/moodys/MoodysRating,Instrument/internal_r/InternalRating,Instrument/sp/SPRating,Instrument/ratings-scope/PreferredRating,Instrument/ratings-scope/AverageRating
0,GB0002374006,Diageo,BBB+,A-,BBB,A+,BBB+,71.666667
1,GB00BMJ6DW54,Informa,AA-,NaN,BBB-,BBB,AA-,72.500000
2,GB00BJFFLV09,Croda International,BBB-,BBB-,NaN,NaN,BBB-,55.000000
3,GB00BYX91H57,JD Sports,AA,A-,NaN,A+,AA,80.000000
4,GB00B3MBS747,Ocado Group,A,A+,A-,A,A,76.666667
...,...,...,...,...,...,...,...,...
95,GB00B1FH8J72,Severn Trent,A+,AA-,AA-,BBB+,A+,76.666667
96,GB00B1XZS820,Anglo American plc,AA,BBB+,NaN,AA,AA,81.666667
97,GB0032089863,Next plc,AA-,A-,A+,BBB+,AA-,73.333333
98,GB00B03MLX29,Royal Dutch Shell,AA,BBB+,BBB,AA,AA,81.666667


In [128]:
for item in instruments_properties_response.values.values():
    
    print(item.identifiers["Isin"])
    print(item.name)
    
    for prop in item.properties:
        
        print(prop.key)
                
        if prop.value.label_value:
            
            print(prop.value.label_value)
            
        else:
            print(prop.value.metric_value.value)
        
    

GB00B19NLV48
Experian
Instrument/sp/SPRating
BBB
Instrument/moodys/MoodysRating
BBB+
Instrument/fitch/FitchRating
AA-
Instrument/internal_r/InternalRating
BBB+
Instrument/ratings-scope/AverageRating
68.75
JE00B4T3BW64
Glencore
Instrument/sp/SPRating
AA
Instrument/moodys/MoodysRating
BBB-
Instrument/fitch/FitchRating
BBB
Instrument/internal_r/InternalRating
AA
Instrument/ratings-scope/AverageRating
73.75
GB0007980591
BP
Instrument/sp/SPRating
A-
Instrument/moodys/MoodysRating
AA-
Instrument/fitch/FitchRating
A+
Instrument/internal_r/InternalRating
A-
Instrument/ratings-scope/AverageRating
76.25
GB00BBG9VN75
Aveva
Instrument/sp/SPRating
A-
Instrument/moodys/MoodysRating
A
Instrument/fitch/FitchRating
A-
Instrument/internal_r/InternalRating
AA-
Instrument/ratings-scope/AverageRating
75.0
IE00BWT6H894
Flutter Entertainment
Instrument/moodys/MoodysRating
BBB-
Instrument/internal_r/InternalRating
BBB+
Instrument/ratings-scope/AverageRating
30.0
GB0009223206
Smith & Nephew
Instrument/sp/SPRat

In [67]:
df.columns

Index(['href', 'scope', 'lusid_instrument_id', 'version.effective_from',
       'version.as_at_date', 'name', 'identifiers.Figi', 'identifiers.Isin',
       'identifiers.LusidInstrumentId', 'identifiers.Ticker',
       'identifiers.Sedol', 'identifiers.ClientInternal',
       'properties(properties.0.value.label_valu-Properties)',
       'properties.0.effective_until',
       'properties(properties.1.value.label_valu-Properties)',
       'properties.1.effective_until',
       'properties(properties.2.value.label_valu-Properties)',
       'properties.2.effective_until',
       'properties(properties.3.value.label_valu-Properties)',
       'properties.3.effective_until',
       'properties(properties.4.value.metric_value.valu-Properties)',
       'instrument_definition.dom_ccy',
       'instrument_definition.instrument_type', 'state', 'asset_class',
       'dom_ccy', 'relationships', 'links.0.relation', 'links.0.href',
       'links.0.method',
       'properties(properties.3.value.metric

In [46]:
df_rows = []

for item in instruments_properties_response.values:
    
    row = instruments_properties_response.values[item].to_dict()
        
    df_rows.append(row)
    
    
    

In [48]:
pd.DataFrame(df_rows)

,href,scope,lusid_instrument_id,version,name,identifiers,properties,lookthrough_portfolio,instrument_definition,state,asset_class,dom_ccy,relationships,links
0,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_00003D7X,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",Glencore,"{'Figi': 'BBG001MM1KV4', 'Isin': 'JE00B4T3BW64...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
1,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_YOOKQSH7,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",BT Group,"{'Figi': 'BBG000C05R82', 'Isin': 'GB0030913577...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
2,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_H1EYNE1E,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",Taylor Wimpey,"{'Figi': 'BBG000BF4KL1', 'Isin': 'GB0008782301...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
3,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_I0UPU8LU,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",Barclays,"{'Figi': 'BBG000C04D57', 'Isin': 'GB0031348658...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
4,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_00003E7M,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",RSA Insurance Group,"{'Figi': 'BBG000H3DN90', 'Isin': 'GB00BKKMKR23...","[{'key': 'Instrument/sp/SPRating', 'value': {'...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_OOUHHR98,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",Standard Chartered,"{'Figi': 'BBG000BF2QW8', 'Isin': 'GB0004082847...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
96,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_IU71BRS7,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",United Utilities,"{'Figi': 'BBG000BBFLV5', 'Isin': 'GB00B39J2M42...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
97,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_YEZBWJJB,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",Prudential plc,"{'Figi': 'BBG000BDY322', 'Isin': 'GB0007099541...","[{'key': 'Instrument/sp/SPRating', 'value': {'...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
98,https://stephenlm.lusid.com/api/api/instrument...,default,LUID_H1XAFPXL,"{'effective_from': 0001-01-01 00:00:00+00:00, ...",DCC plc,"{'Figi': 'BBG000BFLBK3', 'Isin': 'IE0002424939...","[{'key': 'Instrument/moodys/MoodysRating', 'va...",None,"{'identifiers': {'lusid_instrument_id': None, ...",Active,Equities,GBP,[],"[{'relation': 'EntitySchema', 'href': 'https:/..."
